In [1]:
from azure.devops.connection import Connection
from msrest.authentication import BasicAuthentication
import pprint

In [2]:
# Fill in with your personal access token and org URL
personal_access_token = 'spov5s7rr5462wmju5lc3jtdgetc4itewtbvut2ts7iifjflnw3a'
organization_url = 'http://gfsvstf:8080/tfs/GFS%20Service%20Mgmt'

In [3]:
# Create a connection to the org
credentials = BasicAuthentication('', personal_access_token)
connection = Connection(base_url=organization_url, creds=credentials)

In [4]:
# Get a client (the "core" client provides access to projects, teams, etc)
core_client = connection.clients.get_core_client()

In [5]:
get_project_response = core_client.get_projects()

In [6]:
tokens_dict = {}
for projects in get_project_response.value:
    tokens_dict[projects.name] = str('$PROJECT:vstfs:///Classification/TeamProject/'+projects.url.split('/')[-1]+':')

In [7]:
tokens_dict

{'GFS-PlatformServices': '$PROJECT:vstfs:///Classification/TeamProject/71c9f668-f9fb-4c5f-85e1-1c904c7083b2:',
 'GNS-Code': '$PROJECT:vstfs:///Classification/TeamProject/04865936-bbdc-434b-83f5-7b2e4ec1fd02:',
 'GNS-EDGE': '$PROJECT:vstfs:///Classification/TeamProject/b810a24a-e975-4fa9-a6f2-b55fccbb5846:',
 'GFSPrM-MOC': '$PROJECT:vstfs:///Classification/TeamProject/00633567-0f14-4663-9575-e38f9680b31c:',
 'GFS-CSIPRM': '$PROJECT:vstfs:///Classification/TeamProject/d5ca8447-9e11-479e-b6da-4aa00231bea9:',
 'GNS-SM': '$PROJECT:vstfs:///Classification/TeamProject/9ea02088-da8f-4747-8462-493b3b15d8cb:',
 'test': '$PROJECT:vstfs:///Classification/TeamProject/74700dfa-514d-4357-b848-3c8e23314a6e:',
 'Task Management': '$PROJECT:vstfs:///Classification/TeamProject/f1be4206-5b1b-4514-b6ca-f153158602ef:',
 'BGIT_test': '$PROJECT:vstfs:///Classification/TeamProject/e9e06476-0325-49c3-a281-813b475e349d:',
 'GFSPrM-PPE': '$PROJECT:vstfs:///Classification/TeamProject/d5dc4b27-2f6f-4fe5-bf14-880deb

In [8]:
security_client  = connection.clients.get_security_client()

In [9]:
project_security_namespaceid = '52D39943-CB85-4D7F-8FA8-C6BAAC873819'

In [10]:
#Get ACLs for all the projects in the project namespace:-
project_acls = {}
for key in tokens_dict:
    project_acls[key] = security_client.query_access_control_lists(security_namespace_id= project_security_namespaceid, token=tokens_dict[key])

In [11]:
project_acls

{'GFS-PlatformServices': [<azure.devops.v5_1.security.models.AccessControlList at 0x207364bcdc8>],
 'GNS-Code': [<azure.devops.v5_1.security.models.AccessControlList at 0x207364c6f08>],
 'GNS-EDGE': [<azure.devops.v5_1.security.models.AccessControlList at 0x207364cccc8>],
 'GFSPrM-MOC': [<azure.devops.v5_1.security.models.AccessControlList at 0x207364d6cc8>],
 'GFS-CSIPRM': [<azure.devops.v5_1.security.models.AccessControlList at 0x207364dca08>],
 'GNS-SM': [<azure.devops.v5_1.security.models.AccessControlList at 0x207364d9108>],
 'test': [<azure.devops.v5_1.security.models.AccessControlList at 0x207364d2d08>],
 'Task Management': [<azure.devops.v5_1.security.models.AccessControlList at 0x207364c6d88>],
 'BGIT_test': [<azure.devops.v5_1.security.models.AccessControlList at 0x207364bc1c8>],
 'GFSPrM-PPE': [<azure.devops.v5_1.security.models.AccessControlList at 0x207364ed588>]}

In [12]:
sample_project = 'BGIT_test'

In [13]:
sample_project_acl = project_acls[sample_project][0]

In [14]:
sample_project_acl.as_dict()

{'aces_dictionary': {'Microsoft.TeamFoundation.ServiceIdentity;d5df3285-17cc-4f04-8be9-6d20c3217dd5:Build:d17714fd-5a56-484a-99e9-1163029a6352': {'allow': 4777,
   'deny': 0,
   'descriptor': 'Microsoft.TeamFoundation.ServiceIdentity;d5df3285-17cc-4f04-8be9-6d20c3217dd5:Build:d17714fd-5a56-484a-99e9-1163029a6352'},
  'Microsoft.TeamFoundation.Identity;S-1-9-1551374245-1194710491-279448905-2359889803-2671216909-0-0-0-0-1': {'allow': 15989759,
   'deny': 0,
   'descriptor': 'Microsoft.TeamFoundation.Identity;S-1-9-1551374245-1194710491-279448905-2359889803-2671216909-0-0-0-0-1'},
  'Microsoft.TeamFoundation.Identity;S-1-9-1551374245-1194710491-279448905-2359889803-2671216909-1-1077864093-2880561487-2695991113-3413047478': {'allow': 513,
   'deny': 0,
   'descriptor': 'Microsoft.TeamFoundation.Identity;S-1-9-1551374245-1194710491-279448905-2359889803-2671216909-1-1077864093-2880561487-2695991113-3413047478'},
  'Microsoft.TeamFoundation.Identity;S-1-9-1551374245-1194710491-279448905-23598

In [15]:
import json

In [16]:
acl_json = json.dumps(sample_project_acl.as_dict(), indent=2)

In [18]:
with open('acls_bgit_test.json', 'w') as fp:
    json.dump(sample_project_acl.as_dict(), fp)

In [31]:
identity_client.read_identities(descriptors='Microsoft.TeamFoundation.Identity;S-1-9-1551374245-1204400969-2402986413-2179408616-0-0-0-1-2')[0]

In [21]:
id = identity_client.read_identities(descriptors='Microsoft.TeamFoundation.ServiceIdentity;d5df3285-17cc-4f04-8be9-6d20c3217dd5:Build:d17714fd-5a56-484a-99e9-1163029a6352')

In [23]:
id = id[0]

In [32]:
id.as_dict()

{'custom_display_name': 'Project Collection Build Service (TEAM FOUNDATION)',
 'descriptor': 'Microsoft.TeamFoundation.ServiceIdentity;d5df3285-17cc-4f04-8be9-6d20c3217dd5:Build:d17714fd-5a56-484a-99e9-1163029a6352',
 'id': '9090cc59-d8b9-44c1-8a2c-d44e61de258b',
 'is_active': True,
 'master_id': 'ffffffff-ffff-ffff-ffff-ffffffffffff',
 'member_ids': [],
 'member_of': [],
 'members': [],
 'meta_type_id': 255,
 'properties': {'SchemaClassName': {'$type': 'System.String',
   '$value': 'User'},
  'Description': {'$type': 'System.String', '$value': ''},
  'Domain': {'$type': 'System.String', '$value': 'Build'},
  'Account': {'$type': 'System.String',
   '$value': 'd17714fd-5a56-484a-99e9-1163029a6352'},
  'DN': {'$type': 'System.String', '$value': ''},
  'Mail': {'$type': 'System.String', '$value': ''},
  'SpecialType': {'$type': 'System.String', '$value': 'Generic'},
  'ComplianceValidated': {'$type': 'System.DateTime',
   '$value': '2020-08-06T00:00:00Z'},
  'DirectoryAlias': {'$type': '